## Better retrieval via "Dense Passage Retrieval"


### Importance of Retrievers

The Retriever has a huge impact on the performance of our overall search pipeline.


### Different types of Retrievers
#### Sparse
Family of algorithms based on counting the occurences of words (bag-of-words) resulting in very sparse vectors with length = vocab size. 

Examples: BM25, TF-IDF  
Pros: Simple, fast, well explainable  
Cons: Relies on exact keyword matches between query and text  
 

#### Dense
These retrievers use neural network models to create "dense" embedding vectors. Within this family there are two different approaches: 

a) Single encoder: Use a **single model** to embed both query and passage.  
b) Dual-encoder: Use **two models**, one to embed the query and one to embed the passage

Recent work suggests that dual encoders work better, likely because they can deal better with the different nature of query and passage (length, style, syntax ...). 

Examples: REALM, DPR, Sentence-Transformers ...
Pros: Captures semantinc similarity instead of "word matches" (e.g. synonyms, related topics ...) 
Cons: Computationally more heavy, initial training of model  


### "Dense Passage Retrieval"

In this Tutorial, we want to highlight one "Dense Dual-Encoder" called Dense Passage Retriever. 
It was introdoced by Karpukhin et al. (2020, https://arxiv.org/abs/2004.04906. 

Original Abstract: 

_"Open-domain question answering relies on efficient passage retrieval to select candidate contexts, where traditional sparse vector space models, such as TF-IDF or BM25, are the de facto method. In this work, we show that retrieval can be practically implemented using dense representations alone, where embeddings are learned from a small number of questions and passages by a simple dual-encoder framework. When evaluated on a wide range of open-domain QA datasets, our dense retriever outperforms a strong Lucene-BM25 system largely by 9%-19% absolute in terms of top-20 passage retrieval accuracy, and helps our end-to-end QA system establish new state-of-the-art on multiple open-domain QA benchmarks."_

Paper: https://arxiv.org/abs/2004.04906  
Original Code: https://fburl.com/qa-dpr 


*Use this [link](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial6_Better_Retrieval_via_DPR.ipynb) to open the notebook in Google Colab.*


## Prepare environment

### Colab: Enable the GPU runtime 
Make sure you enable the GPU runtime to experience decent speed in this tutorial.  
**Runtime -> Change Runtime type -> Hardware accelerator -> GPU**

<img src="https://raw.githubusercontent.com/deepset-ai/haystack/master/docs/img/colab_gpu_runtime.jpg">

In [1]:
# Make sure you have a GPU running
!nvidia-smi

Mon Aug 17 13:26:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack and install the version of torch that works with the colab GPUs
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-a8wgahlf
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-a8wgahlf
  Created wheel for farm-haystack: filename=farm_haystack-0.3.0-cp36-none-any.whl size=98636 sha256=45ee583c751fe35159a3e3827f944c5d2ef1c84568693209f5e8f2259b61ca9c
  Stored in directory: /tmp/pip-ephem-wheel-cache-oj9e4as1/wheels/ab/41/a4/4fbf362de283352078ecb6705c08b6525347aaea2eead2a60c
  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=dea91678b48bd7bd18b6070290f457fab3204cf8a6db50ff5168d8ae0de44765
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built farm-haystack tika
  Found existing installation: farm-haystack 0.3.0
    Uninstalling farm-haystack-0.3.0:
      Successfully uninstalled farm-haystack-0.3.0


In [3]:
from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

## Document Store

FAISS is a library for efficient similarity search on a cluster of dense vectors.
The `FAISSDocumentStore` uses a SQL(SQLite in-memory be default) database under-the-hood
to store the document text and other meta data. The vector embeddings of the text are
indexed on a FAISS Index that later is queried for searching answers.

In [9]:
from haystack.database.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore()

## Cleaning & indexing documents

Similarly to the previous tutorials, we download, convert and index some Game of Thrones articles to our DocumentStore

In [6]:
# Let's first get some files that we want to use
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

08/17/2020 13:37:07 - INFO - haystack.indexing.utils -   Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
100%|██████████| 1095120/1095120 [00:00<00:00, 15326827.22B/s]
08/17/2020 13:37:09 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.628s]
08/17/2020 13:37:11 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.085s]
08/17/2020 13:37:12 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.070s]
08/17/2020 13:37:13 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.010s]
08/17/2020 13:37:14 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.023s]


## Initalize Retriever, Reader,  & Finder

### Retriever

**Here:** We use a `DensePassageRetriever`

**Alternatives:**

- The `ElasticsearchRetriever`with custom queries (e.g. boosting) and filters
- Use `EmbeddingRetriever` to find candidate documents based on the similarity of embeddings (e.g. created via Sentence-BERT)
- Use `TfidfRetriever` in combination with a SQL or InMemory Document store for simple prototyping and debugging

In [7]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store, embedding_model="dpr-bert-base-nq",
                                  do_lower_case=True, use_gpu=True, embed_title=True)

# Important: 
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation. 
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever)

08/17/2020 13:37:29 - INFO - filelock -   Lock 140258232643032 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


08/17/2020 13:37:29 - INFO - filelock -   Lock 140258232643032 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


08/17/2020 13:37:30 - INFO - filelock -   Lock 140258232709248 acquired on /root/.cache/torch/transformers/8fdd0d2838c23f921379f2b0322aecf406cbdaa97ffecc544e3a1d49a7c302bd.6f90756c59007364d7842118056ad653f39f4d340fbe20bcc04037d2a45cb0f7.lock


08/17/2020 13:37:43 - INFO - filelock -   Lock 140258232709248 released on /root/.cache/torch/transformers/8fdd0d2838c23f921379f2b0322aecf406cbdaa97ffecc544e3a1d49a7c302bd.6f90756c59007364d7842118056ad653f39f4d340fbe20bcc04037d2a45cb0f7.lock


08/17/2020 13:37:50 - INFO - filelock -   Lock 140258226192336 acquired on /root/.cache/torch/transformers/d1c705617c02da7a616f4b5a8cb445a7f78e84bc4f9e26378c89901d97e16d78.232fed629becb590e5b2ac6c6124f9d1561ef7a1d17ad0394232dd46a0835002.lock


08/17/2020 13:38:04 - INFO - filelock -   Lock 140258226192336 released on /root/.cache/torch/transformers/d1c705617c02da7a616f4b5a8cb445a7f78e84bc4f9e26378c89901d97e16d78.232fed629becb590e5b2ac6c6124f9d1561ef7a1d17ad0394232dd46a0835002.lock


08/17/2020 13:38:10 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?scroll=5m&size=1000 [status:200 request:0.315s]
08/17/2020 13:38:10 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.177s]
08/17/2020 13:38:10 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.098s]
08/17/2020 13:38:10 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.004s]
08/17/2020 13:38:10 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.006s]
08/17/2020 13:38:10 - INFO - haystack.database.elasticsearch -   Updating embeddings for 2497 docs ...
08/17/2020 13:39:16 - INFO - haystack.retriever.dense -   Embedded 80 / 2497 texts
08/17/2020 13:40:21 - INFO - haystack.retriever.dense -   Embedded 160 / 2497 texts
08/17/2020 13:41:27 - INFO - haystack.retriever.dense -   Embedded 240 / 2497 texts
08/17/2020 13:42:32 - INFO - haystack

### Reader

Similar to previous Tutorials we now initalize our reader.

Here we use a FARMReader with the *deepset/roberta-base-squad2* model (see: https://huggingface.co/deepset/roberta-base-squad2)



#### FARMReader

In [8]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

08/17/2020 14:12:14 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
08/17/2020 14:12:14 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
08/17/2020 14:12:14 - INFO - filelock -   Lock 140258249886688 acquired on /root/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.c6288e0f84ec797ba5c525c923a5bbc479b47c761aded9734a5f6a473b044c8d.lock


08/17/2020 14:12:15 - INFO - filelock -   Lock 140258249886688 released on /root/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.c6288e0f84ec797ba5c525c923a5bbc479b47c761aded9734a5f6a473b044c8d.lock
08/17/2020 14:12:15 - INFO - filelock -   Lock 140258249883776 acquired on /root/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.d045adc91e17ecdf7dc3eeff4c875df94bdf2eb749d72b3ae47ae93f8e85213c.lock


08/17/2020 14:12:25 - INFO - filelock -   Lock 140258249883776 released on /root/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.d045adc91e17ecdf7dc3eeff4c875df94bdf2eb749d72b3ae47ae93f8e85213c.lock


08/17/2020 14:12:31 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
08/17/2020 14:12:39 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
08/17/2020 14:12:40 - INFO - filelock -   Lock 140258225266640 acquired on /root/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock


08/17/2020 14:12:41 - INFO - filelock -   Lock 140258225266640 released on /root/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock


...

08/17/2020 14:12:43 - INFO - filelock -   Lock 140258225266640 acquired on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


08/17/2020 14:12:43 - INFO - filelock -   Lock 140258225266640 released on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


08/17/2020 14:12:44 - INFO - filelock -   Lock 140258235495424 acquired on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.100c88dbe27dbd73822c575274ade4eb2427596ac56e96769249b7512341654d.lock


08/17/2020 14:12:44 - INFO - filelock -   Lock 140258235495424 released on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.100c88dbe27dbd73822c575274ade4eb2427596ac56e96769249b7512341654d.lock


08/17/2020 14:12:45 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
08/17/2020 14:12:45 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
08/17/2020 14:12:45 - INFO - farm.infer -    0 
08/17/2020 14:12:45 - INFO - farm.infer -   /w\
08/17/2020 14:12:45 - INFO - farm.infer -   /'\
08/17/2020 14:12:45 - INFO - farm.infer -   


### Finder

The Finder sticks together reader and retriever in a pipeline to answer our actual questions. 

In [9]:
finder = Finder(reader, retriever)

## Voilà! Ask a question!

In [10]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
prediction = finder.get_answers(question="Who created the Dothraki vocabulary?", top_k_retriever=10, top_k_reader=5)

#prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_retriever=10, top_k_reader=5)
#prediction = finder.get_answers(question="Who is the sister of Sansa?", top_k_retriever=10, top_k_reader=5)

/usr/local/lib/python3.6/dist-packages/elasticsearch/connection/base.py:177: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
08/17/2020 14:12:46 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.159s]
08/17/2020 14:12:46 - INFO - haystack.finder -   Reader is looking for detailed answer in 11154 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.16 Batches/s]


In [12]:
print_answers(prediction, details="minimal")

[   {   'answer': 'David J. Peterson',
        'context': "age for ''Game of Thrones''\n"
                   'The Dothraki vocabulary was created by David J. Peterson '
                   'well in advance of the adaptation. HBO hired the Language '
                   'Creatio'},
    {   'answer': 'David J. Peterson',
        'context': 'orld. The language was developed for the TV series by the '
                   'linguist David J. Peterson, working off the Dothraki words '
                   "and phrases in Martin's novels.\n"
                   ','},
    {   'answer': 'David J. Peterson',
        'context': '\n'
                   '===Valyrian===\n'
                   'David J. Peterson, who created the Dothraki language for '
                   'the first season of the show, was entrusted by the '
                   'producers to design a new '},
    {   'answer': 'Showrunners D. B. Weiss and David Benioff',
        'context': '\n'
                   '===Conception and development=